In [1]:
import numpy as np
import random
import math
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from vmdpy import VMD
from scipy.fftpack import hilbert,fft,ifft
from math import log
import pandas as pd

In [ ]:
data = np.loadtxt('../可利用代码/ball18.txt')
x = data[0:2048]
#df = pd.read_csv('Solar Power Plant Data.csv',header=0, parse_dates=[0],index_col=0)# 读取CSV文件，并设置第一列为日期，作为索引
#print(df.head(10))
#x = df.iloc[:,:].values
print(x)
tau = 0.  # noise-tolerance (no strict fidelity enforcement)
DC = 0  # no DC part imposed
init = 1  # initialize omegas uniformly
tol = 1e-7

In [3]:
#函数
def fun(K, alpha):
    if K < lb[0]:
        K = lb[0]
    if K > ub[0]:
        K = ub[0]
        
    if alpha < lb[1]:
        alpha = lb[1]
    if alpha > ub[1]:
        alpha =ub[1]

    # K = int(position[0])
    # alpha = position[1]
    u, u_hat, omega = VMD(x, alpha, tau, K, DC, init, tol)
    #  
    EP = []
    for i in range(K):
        H = np.abs(hilbert(u[i,:]))
        e1 = []
        for j in range(len(H)):
            p = H[j]/np.sum(H)
            e = -p*log(p,2)
            e1.append(e)
        E = np.sum(e1)  
        EP.append(E)
    s = np.sum(EP)/K
    return s

In [4]:
# 种群初始化函数

def initial(pop, dim, ub, lb):
    X = np.zeros((pop, dim))
    bound = [lb,ub]
    print(bound)
    for i in range(pop):
        for j in range(dim):
            X[i][j] = np.random.uniform(bound[0][j], bound[1][j])
        X[i] = (int(X[i,0]),int(X[i,1]))
    print(X)
    return X, lb, ub

In [5]:
# 边界检查函数


def BorderCheck(X, ub, lb, pop, dim):
    for i in range(pop):
        for j in range(dim):
            if X[i, j] > ub[j]:
                X[i, j] = ub[j]
            elif X[i, j] < lb[j]:
                X[i, j] = lb[j]
    return X

In [6]:
# 计算适应度函数


def CaculateFitness(X, fun):
    pop = X.shape[0]
    fitness = np.zeros((pop, 1))
    for i in range(pop):
        fitness[i] = fun(int(X[i, 0]), int(X[i, 1]))
    return fitness


In [7]:
# 适应度排序


def SortFitness(Fit):
    fitness = np.sort(Fit, axis=0)
    index = np.argsort(Fit, axis=0)
    return fitness, index


In [8]:
# 根据适应度对位置进行排序

def SortPosition(X, index):
    Xnew = np.zeros(X.shape)
    for i in range(X.shape[0]):
        Xnew[i, :] = X[index[i], :]
    return Xnew

In [10]:
import numpy as np
import random

def initialize_population(pop_size, dim, lb, ub):
    population = []
    for _ in range(pop_size):
        individual = [random.uniform(lb, ub) for _ in range(dim)]
        population.append(individual)
    return population

def fitness_function(individual, fun):
    # 根据问题定义适应度函数，用于评估个体的适应度
    return fun(individual)

def selection(population, fitness_values):
    # 使用轮盘赌选择方法选择个体
    total_fitness = sum(fitness_values)
    probabilities = [fitness / total_fitness for fitness in fitness_values]
    selected_indices = np.random.choice(len(population), len(population), p=probabilities)
    selected_population = [population[i] for i in selected_indices]
    return selected_population

def crossover(parent1, parent2):
    # 单点交叉操作，可以根据需要选择其他交叉方式
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutation(individual, mutation_rate, lb, ub):
    # 简单的变异操作，可以根据需要选择其他变异方式
    mutated_individual = individual.copy()
    for i in range(len(mutated_individual)):
        if random.random() < mutation_rate:
            mutated_individual[i] = random.uniform(lb, ub)
    return mutated_individual

def genetic_algorithm(pop_size, dim, lb, ub, max_iter, fun):
    population = initialize_population(pop_size, dim, lb, ub)
    best_fitness = float('inf')
    best_individual = None
    curve = []

    for _ in range(max_iter):
        fitness_values = [fitness_function(individual, fun) for individual in population]

        if min(fitness_values) < best_fitness:
            best_fitness = min(fitness_values)
            best_individual = population[fitness_values.index(best_fitness)]

        curve.append(best_fitness)

        selected_population = selection(population, fitness_values)
        new_population = []

        for _ in range(pop_size // 2):
            parent1, parent2 = random.choices(selected_population, k=2)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutation(child1, mutation_rate=0.1, lb=lb, ub=ub)
            child2 = mutation(child2, mutation_rate=0.1, lb=lb, ub=ub)
            new_population.extend([child1, child2])

        population = new_population

    return best_fitness, best_individual, curve

# 示例问题的适应度函数
def example_fitness_function(individual):
    return sum([x**2 for x in individual])  # 简单的二次函数示例

if __name__ == "__main__":
    pop_size = 10
    dim = 1
    lb = 2   # 下边界
    ub = 10   # 上边界
    max_iter = 40
    result = genetic_algorithm(pop_size, dim, lb, ub, max_iter, example_fitness_function)
    print("最佳适应度:", result[0])
    print("最佳个体:", result[1])

TypeError: unsupported operand type(s) for -: 'list' and 'list'